In [1]:
import pandas as pd

# Path to the Data folder
data_folder_path = 'Data/'

# Read in the files with full paths
movie_df = pd.read_csv(data_folder_path + 'movie.csv')
rating_df = pd.read_csv(data_folder_path + 'rating.csv')
genome_tags_df = pd.read_csv(data_folder_path + 'genome_tags.csv')
tags_df = pd.read_csv(data_folder_path + 'tag.csv')  # Reading the additional tags.csv file

# Display the DataFrames
print("Movies DataFrame:")
print(movie_df.head())  # Show the first few rows
print("\nRatings DataFrame:")
print(rating_df.head())  # Show the first few rows
print("\nGenome Tags DataFrame:")
print(genome_tags_df.head())  # Show the first few rows
print("\nTags DataFrame:")
print(tags_df.head())  # Show the first few rows of the new DataFrame

# Fill missing values
# The fill value should be chosen based on the data type and context of each column
movie_df.fillna('unknown', inplace=True)
rating_df.fillna(0, inplace=True)  # Assuming 0 is a sensible fill value for ratings
genome_tags_df.fillna('unknown', inplace=True)
tags_df.fillna('unknown', inplace=True)  # Filling missing values in the new DataFrame

# Show DataFrames after filling missing values
print("\nMovies DataFrame after filling missing values:")
print(movie_df.head())
print("\nRatings DataFrame after filling missing values:")
print(rating_df.head())
print("\nGenome Tags DataFrame after filling missing values:")
print(genome_tags_df.head())
print("\nTags DataFrame after filling missing values:")
print(tags_df.head())


FileNotFoundError: [Errno 2] No such file or directory: 'Data/movie.csv'

Merging All Dataframes

In [2]:
# Assuming movie_df, rating_df, and tags_df are already loaded and preprocessed

# Step 1: Merge movie_df with rating_df
movie_rating_merged = pd.merge(movie_df, rating_df, on='movieId', how='outer')

# Step 2: Merge the result with tags_df
full_merged_df = pd.merge(movie_rating_merged, tags_df, on=['movieId', 'userId'], how='outer')

# Display the first few rows of the combined DataFrame to verify
print(full_merged_df.head())


NameError: name 'movie_df' is not defined

In [3]:
# Assuming your merged DataFrame is named full_merged_df
full_merged_df = full_merged_df.drop(columns=['timestamp_x', 'timestamp_y'])


NameError: name 'full_merged_df' is not defined

Doing NLP on the genre and tags

In [4]:
full_merged_df

NameError: name 'full_merged_df' is not defined

In [5]:
# Assuming your merged DataFrame is named full_merged_df
nan_counts = full_merged_df.isna().sum()

# Print the counts of NaN values per column
print(nan_counts)

NameError: name 'full_merged_df' is not defined

In [6]:
full_merged_df_cleaned = full_merged_df.dropna()

NameError: name 'full_merged_df' is not defined

In [7]:
full_merged_df_cleaned

NameError: name 'full_merged_df_cleaned' is not defined

Generating Embeddings for The Tags, Genre, and Title. Since These are all text based, theoretically they have some kind of semantic meaning that could be captured

In [8]:
from transformers import BertModel, BertTokenizer
import torch

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load pre-trained model
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()  # Set the model to evaluation mode


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [9]:
# Function to generate embeddings in batches
def generate_embeddings(model, encoded_inputs, batch_size=10):
    # Ensure inputs are on the same device as the model
    device = next(model.parameters()).device
    input_ids = encoded_inputs['input_ids'].to(device)
    attention_mask = encoded_inputs['attention_mask'].to(device)
    
    model.to(device)
    
    embeddings = []
    for i in range(0, len(input_ids), batch_size):
        batch_input_ids = input_ids[i:i+batch_size]
        batch_attention_mask = attention_mask[i:i+batch_size]
        
        with torch.no_grad():
            outputs = model(batch_input_ids, attention_mask=batch_attention_mask)
        
        # Use mean pooling to get a single vector for the sentence
        batch_embeddings = outputs.last_hidden_state.mean(dim=1)
        embeddings.append(batch_embeddings.cpu())
    
    # Concatenate all batch embeddings
    embeddings = torch.cat(embeddings, dim=0)
    return embeddings

# Generate embeddings
embeddings = generate_embeddings(model, encoded_inputs)


NameError: name 'encoded_inputs' is not defined